In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0', ignore_verifications=True)

from datasets import get_dataset_split_names
get_dataset_split_names("cnn_dailymail", '3.0.0')

In [ ]:
import pandas as pd
import re

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn. metrics import adjusted_rand_score
import string

In [ ]:
dataset = load_dataset("cnn_dailymail", '3.0.0', split="train")
dataframe = pd.DataFrame(dataset)
df = dataframe.loc[0:9999]

articles = df["article"].tolist()

In [ ]:
clean_articles = []
for article in articles:
  text = re.sub(r"NEW:", "", article)
  text = text.lower() # нижний регистр
  text_list_nltk = word_tokenize(text) # токенизация
  stop_words = stopwords.words('english')
  text_clean = [word for word in text_list_nltk if word not in stop_words and word[0].isalpha()] # чистим от стоп-слов и пунктуации
  text_clean = ' '.join(text_clean)
  clean_articles.append(text_clean)

In [ ]:
vectorizer = TfidfVectorizer(
                                max_features = 100,
                                max_df = 0.8,
                                min_df = 5,
                                ngram_range = (1,3),
                                stop_words = "english"
                            )

In [ ]:
vectors = vectorizer.fit_transform(clean_articles)

In [ ]:
feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

for description in denselist:
    x = 0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x += 1
    all_keywords.append(keywords)


In [ ]:
true_k = 20

model = KMeans(
    
               n_clusters = true_k, 
               init = "k-means++", 
               max_iter = 100, 
            n_init = 1
)

model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

In [ ]:
with open ("results.txt", "w", encoding = "utf-8") as f:
    for i in range(true_k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write('%s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")